In [1]:
from matplotlib import pyplot as plt
import psycopg2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Paramètres de connexion à la base de données PostgreSQL
conn_params = {
    "host": "localhost",
    "port": "15432", 
    "database": "postgres",
    "user": "admin",
    "password": "admin"
}

# Connexion à la base de données
conn = psycopg2.connect(**conn_params)

# Création d'un curseur pour exécuter des requêtes SQL
cur = conn.cursor()

# Requête SQL pour sélectionner les données de la table économie
query_economie = "SELECT avg_1982, avg_1983, avg_1984, avg_1985, avg_1986, avg_1987, avg_1988, avg_1989, avg_1990, avg_1991, avg_1992, avg_1993, avg_1994, avg_1995, avg_1996, avg_1997, avg_1998, avg_1999, avg_2000, avg_2001, avg_2002, avg_2003, avg_2004, avg_2005, avg_2006, avg_2007, avg_2008, avg_2009, avg_2010, avg_2011, avg_2012, avg_2013, avg_2014, avg_2015, avg_2016, avg_2017, avg_2018, avg_2019, avg_2020, avg_2021, avg_2022, avg_2023, code_postal, id_economie FROM economie;"
cur.execute(query_economie)
rows_economie = cur.fetchall()

# Requête SQL pour sélectionner les données du premier tour (ou du second tour si nécessaire)
query_election = "SELECT code_postal, \"% Voix/Ins 1\", \"% Voix/Ins 2\" FROM election_2022_t2"
cur.execute(query_election)
rows_election = cur.fetchall()

# Fermeture du curseur et de la connexion à la base de données
cur.close()
conn.close()

# Création des DataFrames à partir des résultats des requêtes
df_economie = pd.DataFrame(rows_economie, columns=['avg_1982', 'avg_1983', 'avg_1984', 'avg_1985', 'avg_1986', 'avg_1987', 'avg_1988', 'avg_1989', 'avg_1990', 'avg_1991', 'avg_1992', 'avg_1993', 'avg_1994', 'avg_1995', 'avg_1996', 'avg_1997', 'avg_1998', 'avg_1999', 'avg_2000', 'avg_2001', 'avg_2002', 'avg_2003', 'avg_2004', 'avg_2005', 'avg_2006', 'avg_2007', 'avg_2008', 'avg_2009', 'avg_2010', 'avg_2011', 'avg_2012', 'avg_2013', 'avg_2014', 'avg_2015', 'avg_2016', 'avg_2017', 'avg_2018', 'avg_2019', 'avg_2020', 'avg_2021', 'avg_2022', 'avg_2023', 'code_postal', 'id_economie'])
df_election = pd.DataFrame(rows_election, columns=['code_postal', '% Voix/Ins 1', '% Voix/Ins 2'])

# Fusionner les données économiques avec les données de l'élection sur la colonne code_postal
df_merged = pd.merge(df_economie, df_election, on='code_postal')

# Sélectionner uniquement les données pour La République En Marche
df_party = df_merged

# Ajout d'une colonne 'Résultat' pour indiquer Gagné (1) ou Perdu (0)
df_party['Résultat'] = df_party['% Voix/Ins 1'].apply(lambda x: 1 if x > 50 else 0)

# Séparer les features (X) et la target (y)
X = df_party.drop(['code_postal', 'id_economie', '% Voix/Ins 1', '% Voix/Ins 2', 'Résultat'], axis=1)
y = df_party['Résultat']

# Diviser les données en ensemble d'entraînement et ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner le modèle de régression logistique multinomiale sur l'ensemble d'entraînement
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Prédire les résultats sur l'ensemble de test pour évaluer la performance du modèle
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Précision du modèle sur l'ensemble de test:", accuracy)
    
# Utiliser le modèle entraîné pour prédire le résultat de La République En Marche dans le département 89
dept_89_economie = df_economie[df_economie['code_postal'] == 89].drop(['code_postal', 'id_economie'], axis=1)
prediction_dept_89 = model.predict(dept_89_economie)
if prediction_dept_89[0] == 1:
    print("Prédiction du vote de La République En Marche dans le département 89: Gagné")
else:
    print("Prédiction du vote de La République En Marche dans le département 89: Perdu")


Précision du modèle sur l'ensemble de test: 0.9001420136034083
Prédiction du vote de La République En Marche dans le département 89: Perdu
